<img align="right" width="250" height="150" src="https://lh3.googleusercontent.com/p/AF1QipPWZQfa087JiVjutpUTVEGRnh6W214Wjm439gKQ=w1080-h608-p-no-v0">

## **Proyecto:** Retención de usuarios de la Plataforma Digital

#### El objetivo de este notebook es incorporar como datos agregados al archivo de Crosseling, los datos de Usuarios, Movimientos, Reclamos y Turnos

## Importación de librerias y datos globales

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import os

try:
    # Monto la unidad de drive porque los archivos son muy grandes
    from google.colab import drive
    drive.mount('/content/drive')
    DIR = '/content/drive/MyDrive/Trayecto3-DataDriven/Data/'
except:
    # Sino lo consigo asumo que estoy trabajando local
    DIR = os.path.abspath(r'..\Data\Export') + '\\'

PERIODOS = ['202202', '202203', '202204']

## Lectura de archivo de usuarios

In [2]:
file = DIR + 'Usuarios.parquet'
df_usuarios = pd.read_parquet(file)

In [3]:
# Filtro los registros que se crearon hace menos de tres meses ya que todos ellos no se pueden evaluar
df_usuarios = df_usuarios[df_usuarios['FechaCreacion'] < dt.datetime(2022, 2, 1)]

In [4]:
fecha_corte =  dt.datetime(2022, 4, 30)
df_usuarios[df_usuarios['LAST_LOGIN'] > fecha_corte] = fecha_corte

In [5]:
# Calculo la cantidad de dias desde su último acceso a la plataforma 
df_usuarios['DIAS_SIN_USAR_PD'] = (fecha_corte - df_usuarios['LAST_LOGIN']).astype('timedelta64[D]').astype(int)

In [6]:

# Calculo Antiguedad en dias la plataforma
df_usuarios['ANTIGUEDAD_PD'] = (df_usuarios['LAST_LOGIN'] - df_usuarios['FechaCreacion']).astype('timedelta64[D]').astype(int)
# Filtro los registros sin antiguedad en la plataforma
df_usuarios = df_usuarios.loc[df_usuarios['ANTIGUEDAD_PD'] > 0]

In [7]:
# Asumo que abandonó la PD si lleva mas de 90 dias sin hacer login
df_usuarios["ABANDONO_PD"] = ['Sí' if ( _ > 90 ) else 'No' for _ in df_usuarios["DIAS_SIN_USAR_PD"]]

In [8]:
df_usuarios.sample(5)

,ID_USER,FechaCreacion,LAST_LOGIN,SegFactor,FIRST_2_DIGITS_DNI,DIAS_SIN_USAR_PD,ANTIGUEDAD_PD,ABANDONO_PD
121979,ac744f4dcd90403fb87401a439b946ad,2020-07-24,2021-08-10,SMS,04,263,382,Sí
143086,248577211cd045a29e21697b6161eaaa,2021-05-10,2021-11-01,Softoken,42,180,175,Sí
48624,4b721951474c473e81f938825550d131,2020-06-18,2020-09-08,SMS,10,599,82,Sí
150091,211afad6b3b24e71b92571bb6d505397,2021-03-04,2021-11-26,SMS,18,155,267,Sí
127621,a2d0dd13339f41d4981d1fe9aace414b,2020-05-30,2021-09-04,SMS,35,238,462,Sí


## Lectura de archivo de Crosseling

Se lee el archivo de crosseling, se filtra para individuos y se cruza con el archivo de usuarios

In [9]:
def tratar_crosseling(periodo: str, df_usr: pd.DataFrame):

  file = DIR + 'Crosseling_{}.parquet'.format(periodo)
  df_crosseling = pd.read_parquet(file)

  # Solo me quedo con los 'Individuo sin actividad comercial'
  df_crosseling = df_crosseling.loc[df_crosseling['GRUPOGENERAL'] == 'Individuo sin actividad comercial']
  df_crosseling.drop('GRUPOGENERAL', axis=1, inplace=True)

  # Cruzar con el archivo de usuarios y retornar
  return pd.merge(df_usuarios, df_crosseling, how='inner', on = 'ID_USER')

## Proceso de incorporación de información de Movimientos

El objeto de leer el archivo de movimientos es incorporar a la tabla de crosseling nuevas columnas con información de:

- Cantidad de movimientos monetarios y monto total realizados por medio de la plataforma digital
- Cantidad de movimientos monetarios y monto total realizados por FUERA de la plataforma digital
- Cantidad de movimientos NO monetarios y realizados por medio de la plataforma digital
- Cantidad de movimientos NO monetarios y realizados por FUERA de la plataforma digital

Total **6** nuevas variables

La idea es tener es que el algoritmo de predicción elegido tenga información adicional para poder inferir si el cliente abandona o no la plataforma

In [10]:
def tratar_movimientos(periodo: str, df_cons: pd.DataFrame):

    uc = [
            'ID_USER',
            'CANAL_RATIO',
            'CANT_OPERACIONES',
            'MONTO_TOTAL'
        ]

    df_estudio = df_cons
    
    #### Transacciones Monetarias ####

    file = DIR + r'Sum_Trx_Monetarias_{}.parquet'.format(periodo)
    df_tx = pd.read_parquet(file, columns=uc)

    # Agrupo y sumarizo los registros por canales digitales y no digitales
    df_tx = df_tx.groupby([
        'ID_USER',
        'CANAL_RATIO']) \
        .aggregate('sum') \
        .sort_values(by='ID_USER') \
        .reset_index()

    # Pongo en columnas la cantidad y monto de las transacciones por canal digital
    df_tx_join = df_tx \
        .loc[(df_tx['CANAL_RATIO']=='DIGITAL'), ['ID_USER', 'CANT_OPERACIONES', 'MONTO_TOTAL']] \
        .rename(columns = {'CANT_OPERACIONES': 'CANT_OP_MON_DIG', 'MONTO_TOTAL': 'MONT_OP_MON_DIG'})
    df_estudio = pd.merge(df_estudio, df_tx_join, how='left', on = 'ID_USER', indicator=False)

    # Pongo en columnas la cantidad y monto de las transacciones por canal NO digital
    df_tx_join = df_tx \
        .loc[(df_tx['CANAL_RATIO']!='DIGITAL'), ['ID_USER', 'CANT_OPERACIONES', 'MONTO_TOTAL']] \
        .rename(columns = {'CANT_OPERACIONES': 'CANT_OP_MON_OTR', 'MONTO_TOTAL': 'MONT_OP_MON_OTR'})
    df_estudio = pd.merge(df_estudio, df_tx_join, how='left', on = 'ID_USER', indicator=False)

    #### Transacciones No Monetarias ####
    file = DIR + r'Sum_Trx_NoMonetarias_{}.parquet'.format(periodo)
    df_tx = pd.read_parquet(file, columns=uc)

    # Agrupo los registros por canales digitales y no digitales
    df_tx = df_tx.groupby([
        'ID_USER',
        'CANAL_RATIO']) \
        .aggregate('sum') \
        .sort_values(by='ID_USER') \
        .reset_index()
        
    # Pongo en columnas la cantidad y monto de las transacciones por canal digital
    df_tx_join = df_tx \
        .loc[(df_tx['CANAL_RATIO']=='DIGITAL'), ['ID_USER', 'CANT_OPERACIONES']] \
        .rename(columns = {'CANT_OPERACIONES': 'CANT_OP_NOMON_DIG'})
    df_estudio = pd.merge(df_estudio, df_tx_join, how='left', on = 'ID_USER', indicator=False)

    # Pongo en columnas la cantidad y monto de las transacciones por canal NO digital
    df_tx_join = df_tx \
        .loc[(df_tx['CANAL_RATIO']!='DIGITAL'), ['ID_USER', 'CANT_OPERACIONES']] \
        .rename(columns = {'CANT_OPERACIONES': 'CANT_OP_NOMON_OTR'})
    df_estudio = pd.merge(df_estudio, df_tx_join, how='left', on = 'ID_USER', indicator=False)

    # Convertir las columnas de cantidades a enteros previo sacar los valores nulos
    for i in ['CANT_OP_MON_DIG', 'CANT_OP_MON_OTR', 'CANT_OP_NOMON_DIG', 'CANT_OP_NOMON_OTR']:
        df_estudio[i] = df_estudio[i].fillna(0)
        df_estudio[i] = df_estudio[i].astype(int)
        
    # Eliminar los nulos de los montos
    for i in ['MONT_OP_MON_DIG', 'MONT_OP_MON_OTR']:
        df_estudio[i] = df_estudio[i].fillna(0)

    return df_estudio


## Proceso de incorporación de información de Reclamos

El objeto de leer el archivo de reclamos es incorporar a la tabla de crosseling una nueva columnas con información de:

- Cantidad de tickets registrados para la plataforma BANCON

In [11]:
def tratar_reclamos(periodo: str, df_cons: pd.DataFrame):

    file = DIR + 'Gestar_{}.parquet'.format(periodo)

    df_gt = pd.read_parquet(file, columns=['ID_USER', 'TYPE MOTIVO GESTION BANCON'])

    # Me quedo solo con lo que son Reclamos de BANCON y los cuento por usuario
    df_gt = df_gt[df_gt['TYPE MOTIVO GESTION BANCON'].notnull()]
    df_gt = df_gt.groupby(by=['ID_USER']).size().to_frame().reset_index()
    df_gt.columns = ['ID_USER', 'CANT_REC_PD']

    df_estudio = pd.merge(df_cons, df_gt, how='left', on='ID_USER', indicator=False)

    # Convertir las columnas de cantidades a enteros previo sacar los valores nulos
    df_estudio['CANT_REC_PD'] = df_estudio['CANT_REC_PD'].fillna(0)
    df_estudio['CANT_REC_PD'] = df_estudio['CANT_REC_PD'].astype(int)

    return df_estudio


## Proceso Principal de incorporación de información al dataframe de Crosseling

In [12]:
for periodo in PERIODOS:
    
    df_parcial = tratar_crosseling(periodo, df_usuarios)
    df_parcial = tratar_movimientos(periodo, df_parcial)
    df_parcial = tratar_reclamos(periodo, df_parcial)
    
    #### Grabar ####
    file = DIR + 'Crosseling_ampliado_{}.parquet'.format(periodo)
    df_parcial.to_parquet(file)
    print(file)

c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Crosseling_ampliado_202202.parquet
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Crosseling_ampliado_202203.parquet
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Crosseling_ampliado_202204.parquet
